In [1]:
import pandas as pd
import numpy as np
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# # read and display results
# results = pd.read_csv("/kaggle/input/international-football-results-from-1872-to-2017/results.csv")
# results.head(10)
results = pd.read_csv('./results/results.csv')

In [3]:
results.head(10)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False
5,1876-03-25,Scotland,Wales,4,0,Friendly,Glasgow,Scotland,False
6,1877-03-03,England,Scotland,1,3,Friendly,London,England,False
7,1877-03-05,Wales,Scotland,0,2,Friendly,Wrexham,Wales,False
8,1878-03-02,Scotland,England,7,2,Friendly,Glasgow,Scotland,False
9,1878-03-23,Scotland,Wales,9,0,Friendly,Glasgow,Scotland,False


In [4]:
# Check for types of columns
results.dtypes

date          object
home_team     object
away_team     object
home_score     int64
away_score     int64
tournament    object
city          object
country       object
neutral         bool
dtype: object

### Transform date columns to a date

In [5]:
results['date'] = pd.to_datetime(results['date'])

### Create columns for year, month and day

In [6]:
results['year'] = results['date'].apply(lambda x : x.year)
results['month'] = results['date'].apply(lambda x : x.month)
results['day'] = results['date'].apply(lambda x : x.day)

### Creat home_team_wins and away_team_wins and draw columns

In [7]:
results['home_team_wins'] = (results['home_score'] - results['away_score']) > 0
results['away_team_wins'] = (results['home_score'] - results['away_score']) < 0
results['draw'] = (results['home_score'] - results['away_score']) == 0

### Store home teams and away teams in lists


In [8]:
home_teams = results['home_team'].unique()
away_teams = results['away_team'].unique()

### tournaments, cities and countries

In [9]:
tournaments = results['tournament'].unique()
cities = results['city'].unique()
countries = results['country'].unique()

### See results of Morocco


In [10]:
results_morocco = results.loc[(results.home_team == 'Morocco') | (results.away_team == 'Morocco'), :]
results_morocco

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year,month,day,home_team_wins,away_team_wins,draw
4328,1958-04-03,Morocco,Libya,3,1,Friendly,Casablanca,Morocco,False,1958,4,3,True,False,False
4656,1960-01-01,Morocco,Yugoslavia,0,5,Friendly,Casablanca,Morocco,False,1960,1,1,False,True,False
4842,1960-10-30,Morocco,Tunisia,2,1,FIFA World Cup qualification,Casablanca,Morocco,False,1960,10,30,True,False,False
4851,1960-11-13,Tunisia,Morocco,2,1,FIFA World Cup qualification,Tunis,Tunisia,False,1960,11,13,True,False,False
4865,1960-12-11,Morocco,German DR,2,3,Friendly,Casablanca,Morocco,False,1960,12,11,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43327,2022-03-25,DR Congo,Morocco,1,1,FIFA World Cup qualification,Kinshasa,DR Congo,False,2022,3,25,False,False,True
43386,2022-03-29,Morocco,DR Congo,4,1,FIFA World Cup qualification,Casablanca,Morocco,False,2022,3,29,True,False,False
43475,2022-06-01,United States,Morocco,3,0,Friendly,Cincinnati,United States,False,2022,6,1,True,False,False
43624,2022-06-09,Morocco,South Africa,2,1,African Cup of Nations qualification,Rabat,Morocco,False,2022,6,9,True,False,False


### Get results of a specific two countries

In [11]:
def get_results_of_two_countries(results_, country1, country2):
    results_of_two_countries = results_.loc[((results_.home_team == country1) & (results_.away_team == country2)) 
                                           | ((results_.home_team == country2) & (results_.away_team == country1)), :]
    return results_of_two_countries

### Get results for Morocco and Brazil

In [12]:
get_results_of_two_countries(results, 'Morocco', 'Brazil')

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year,month,day,home_team_wins,away_team_wins,draw
21037,1997-10-09,Brazil,Morocco,2,0,Friendly,Belém,Brazil,False,1997,10,9,True,False,False
21496,1998-06-16,Brazil,Morocco,3,0,FIFA World Cup,Nantes,France,True,1998,6,16,True,False,False


### Define a function that return historical probabilies of win, loose and draw for two countries

In [13]:
def get_hist_proba_of_two_countries(results_, country1, country2):
    probas = dict()
    # get results of country1 and country2
    temp = get_results_of_two_countries(results_, country1, country2)
    temp = temp[['home_team', 'away_team', 'home_team_wins', 'away_team_wins',  'draw']]
    temp = temp.groupby(['home_team', 'away_team']).sum()
    
    probas[(country1, country2)] = {'Win' : 0, 'Loose' : 0, 'Draw' : 0, 'Games' : 0 }
    temp
    
    if len(temp) == 2: # games are played in country1 and country2
        probas[(country1, country2)]['Win'] = temp.loc[(country1, country2)]['home_team_wins'] + temp.loc[(country2, country1)]['away_team_wins']
        probas[(country1, country2)]['Loose'] = temp.loc[(country1, country2)]['away_team_wins'] + temp.loc[(country2, country1)]['home_team_wins']
        probas[(country1, country2)]['Draw'] = temp.loc[(country1, country2)]['draw'] + temp.loc[(country2, country1)]['draw']
        n_games = probas[(country1, country2)]['Win'] + probas[(country1, country2)]['Loose'] + probas[(country1, country2)]['Draw']
        
        if n_games > 0 :
            probas[(country1, country2)]['Win'] = probas[(country1, country2)]['Win']/n_games
            probas[(country1, country2)]['Loose'] = probas[(country1, country2)]['Loose']/n_games
            probas[(country1, country2)]['Draw'] = probas[(country1, country2)]['Draw']/n_games
            probas[(country1, country2)]['Games'] = n_games
            
    
    if len(temp) == 1: # games are played in one country only
        if (country1, country2) in temp.index: # all games were played in country1, so use (country1, country2) as index for temp
            probas[(country1, country2)]['Win'] = temp.loc[(country1, country2)]['home_team_wins']
            probas[(country1, country2)]['Loose'] = temp.loc[(country1, country2)]['away_team_wins']
            probas[(country1, country2)]['Draw'] = temp.loc[(country1, country2)]['draw']
            n_games = probas[(country1, country2)]['Win'] + probas[(country1, country2)]['Loose'] + probas[(country1, country2)]['Draw']
        
            if n_games > 0 :
                probas[(country1, country2)]['Win'] = probas[(country1, country2)]['Win']/n_games
                probas[(country1, country2)]['Loose'] = probas[(country1, country2)]['Loose']/n_games
                probas[(country1, country2)]['Draw'] = probas[(country1, country2)]['Draw']/n_games
                probas[(country1, country2)]['Games'] = n_games
        else: # all games were played in country2, so use (country2, country1) as index for temp
            probas[(country1, country2)]['Win'] = temp.loc[(country2, country1)]['away_team_wins']
            probas[(country1, country2)]['Loose'] = temp.loc[(country2, country1)]['home_team_wins']
            probas[(country1, country2)]['Draw'] = temp.loc[(country2, country1)]['draw']
            n_games = probas[(country1, country2)]['Win'] + probas[(country1, country2)]['Loose'] + probas[(country1, country2)]['Draw']
        
            if n_games > 0 :
                probas[(country1, country2)]['Win'] = probas[(country1, country2)]['Win']/n_games
                probas[(country1, country2)]['Loose'] = probas[(country1, country2)]['Loose']/n_games
                probas[(country1, country2)]['Draw'] = probas[(country1, country2)]['Draw']/n_games
                probas[(country1, country2)]['Games'] = n_games
                                                    
       
    return probas
    

### historical probabilities of Argentina and Brazil

In [14]:
argentina_brazil = get_hist_proba_of_two_countries(results,  'Argentina', 'Brazil')
argentina_brazil

{('Argentina', 'Brazil'): {'Win': 0.3611111111111111,
  'Loose': 0.39814814814814814,
  'Draw': 0.24074074074074073,
  'Games': 108}}

### Define a function that returns historical probabilities for each pair of two countries
### from a given set of results

In [15]:
def get_hist_proba(results_): # not using results to avoid ambiguity !
    # get all pairs of countries having played a match
    matches = list(results_[['home_team', 'away_team', 'home_score']].groupby(['home_team', 'away_team']).sum().index)
    
    # get ride of (country2, country1) if (country1, country2) already exist in the list
    matches2 = [] # new list after removing duplicates
    for m in matches:
        if ((m[0], m[1]) in matches2) or ((m[1], m[0]) in matches2):
            continue
        else:
            matches2.append(m)
    
    countries1 = [] # first country : country1
    countries2 = [] # second country : country2
    games = [] # number of games played by that pair of countries (country1, country2)
    wins = []  # number of games wined by country1 for the pair (country1, country2)
    looses = [] # number of games lost by country1 for the pair (country1, country2)
    draws = [] # number of draws for the pair (country1, country2)
    for m in matches2:
        temp = get_hist_proba_of_two_countries(results_,  m[0], m[1])
        # we have to add two raws : one for country1, country2 and the other for country2, country1
        # country1, country2
        countries1.append(m[0])
        countries2.append(m[1])
        games.append(temp[m]['Games'])
        wins.append(temp[m]['Win'])
        looses.append(temp[m]['Loose'])
        draws.append(temp[m]['Draw'])
        # country2, country1
        countries1.append(m[1])
        countries2.append(m[0])
        games.append(temp[m]['Games']) # games played is the same
        wins.append(temp[m]['Loose']) # for win and loose we have to switch !
        looses.append(temp[m]['Win']) # for win and loose we have to switch !
        draws.append(temp[m]['Draw']) # draw  is the same
        
    
    historical_ratios = pd.DataFrame({'country1' : countries1 , 'country2' : countries2 ,'games' : games,'wins' : wins, 'looses' : looses,  'draws' : draws})

    historical_ratios = historical_ratios.set_index(['country1', 'country2'])

    return historical_ratios
    

In [16]:
# Read qatar 2022 teams file
qatar_2022_teams = pd.read_csv('./qatar2022/Qatar2022-teams.csv',sep=';')
qatar_2022_teams

,Team,Group
0,Senegal,A
1,Qatar,A
2,Netherlands,A
3,Ecuador,A
4,Iran,B
5,England,B
6,USA,B
7,Wales,B
8,Argentina,C
9,Saudi Arabia,C


### get results for these teams

In [22]:
# get results for these teams
qatar_2022_teams_list = list(qatar_2022_teams['Team'])
qatar_2022_results = results.loc[(results.home_team.isin(qatar_2022_teams_list)) & (results.away_team.isin(qatar_2022_teams_list)),: ]

qatar_2022_results

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year,month,day,home_team_wins,away_team_wins,draw
10,1879-01-18,England,Wales,2,1,Friendly,London,England,False,1879,1,18,True,False,False
14,1880-03-15,Wales,England,2,3,Friendly,Wrexham,Wales,False,1880,3,15,False,True,False
16,1881-02-26,England,Wales,0,1,Friendly,Blackburn,England,False,1881,2,26,False,True,False
22,1882-03-13,Wales,England,5,3,Friendly,Wrexham,Wales,False,1882,3,13,True,False,False
24,1883-02-03,England,Wales,5,0,Friendly,London,England,False,1883,2,3,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43695,2022-06-12,Switzerland,Portugal,1,0,UEFA Nations League,Geneva,Switzerland,False,2022,6,12,True,False,False
43713,2022-06-13,France,Croatia,0,1,UEFA Nations League,Paris,France,False,2022,6,13,False,True,False
43739,2022-06-14,Netherlands,Wales,3,2,UEFA Nations League,Rotterdam,Netherlands,False,2022,6,14,True,False,False
43740,2022-06-14,Poland,Belgium,0,1,UEFA Nations League,Warsaw,Poland,False,2022,6,14,False,True,False


## Get historical probabilities of teams participating in 2022 World Cup

In [18]:
hist_proba_qatar_teams = get_hist_proba(qatar_2022_results)
hist_proba_qatar_teams.reset_index().to_csv('./qatar2022/historical_win-loose-draw_ratios_qatar2022_teams.csv', index=None)
hist_proba_qatar_teams

,,games,wins,looses,draws
country1,country2,,,,
Argentina,Australia,7,0.714286,0.142857,0.142857
Australia,Argentina,7,0.142857,0.714286,0.142857
Argentina,Belgium,4,0.750000,0.250000,0.000000
Belgium,Argentina,4,0.250000,0.750000,0.000000
Argentina,Brazil,108,0.361111,0.398148,0.240741
...,...,...,...,...,...
Australia,Wales,1,1.000000,0.000000,0.000000
Wales,Costa Rica,2,0.500000,0.500000,0.000000
Costa Rica,Wales,2,0.500000,0.500000,0.000000


In [19]:
# Check for Argentina and Brazil - we can search for ('Argentina', 'Brazil') or ('Brazil', 'Argentina')
# and it should give the same results !
print ( 'Probabiliy Argentina wins : ' , hist_proba_qatar_teams.loc[('Argentina', 'Brazil')]['wins'])
print ( 'Probabiliy Brazil wins : ' , hist_proba_qatar_teams.loc[('Argentina', 'Brazil')]['looses'])
print ( 'Probabiliy of draw : ' , hist_proba_qatar_teams.loc[('Argentina', 'Brazil')]['draws'])
print("-----------------------------------------------------------------------------------------------------------")
print ( 'Probabiliy Argentina wins : ' , hist_proba_qatar_teams.loc[('Brazil', 'Argentina')]['looses'])
print ( 'Probabiliy Brazil wins : ' , hist_proba_qatar_teams.loc[('Brazil', 'Argentina')]['wins'])
print ( 'Probabiliy of draw : ' , hist_proba_qatar_teams.loc[('Brazil', 'Argentina')]['draws'])


Probabiliy Argentina wins :  0.3611111111111111
Probabiliy Brazil wins :  0.39814814814814814
Probabiliy of draw :  0.24074074074074073
-----------------------------------------------------------------------------------------------------------
Probabiliy Argentina wins :  0.3611111111111111
Probabiliy Brazil wins :  0.39814814814814814
Probabiliy of draw :  0.24074074074074073


In [20]:
hist_proba_qatar_teams.reset_index().to_csv('./hist_proba_qatar_teams.csv', index=None)